<a href="https://colab.research.google.com/github/SamuelXJames/Signal-Denoising-Autoencoder/blob/master/CNN1D_Optimize%20v0.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#Additional Packages to Install
# !pip install pydrive
# !pip install git+https://github.com/hyperopt/hyperopt.
# !pip install kopt
# !pip install oauth2client
# !pip install tensorboardcolab

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pydrive
from tensorboardcolab import *
from random import randint
from keras.layers import Dense,Conv1D,Input
from keras.layers import MaxPooling1D,UpSampling1D
from keras.layers import Activation,BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from kopt import CompileFN, KMongoTrials, test_fn
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


#Authenticate/Create PyDrive client 
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

#Create File in Collab
#model_file = drive.CreateFile({'title' : 'backup.json'})
#model_file.SetContentFile('model.h5')

  
#tbc=TensorBoardColab()
batch_size = 1000
validation_split = 0.15
timesteps = 2000
features = 1
global timesteps
global fe#atures


#Generate Test/Train Data
def create_data(timesteps = 2000):
  length = 1
  dt = 0.0005
  noise_factor = 0.35
  data_1 = []
  data_noisy_1 = []
  for i in range(batch_size):
      freq = 30*np.random.random() + 1
      t = np.arange(0,length,dt)
      #signal = np.arange(-0.5*length,0.5*length,dt)
      signal = np.sin(2*np.pi*freq*t)
      noisy_signal = signal+noise_factor*np.random.rand(len(t))
      data_1.append(signal)
      data_noisy_1.append(noisy_signal)
  data_1 = np.array(data_1)
  data_noisy_1 = np.array(data_noisy_1)
  
  #Scale Data - Need a better scaler than MinMaxScaler
  scaler = MinMaxScaler(feature_range = (0,1))
  data_1  = np.vstack((data_1,data_noisy_1))
  data_1 = scaler.fit_transform(data_1)
  signal = data_1[0:batch_size][:]
  signal_noisy = data_1[batch_size:2*batch_size][:]
  
  #Input Shaping
  features = 1
  timesteps = len(signal[0])
  shape = (batch_size,timesteps,features)
  #Input shape should equal [samples, timesteps, features]
  x = signal.reshape(shape)
  y = signal_noisy.reshape(shape)
  
  split_point = int((1-validation_split)*batch_size)
  
  x_train = x[0:split_point,:,:]
  y_train = y[0:split_point,:,:]
  x_test = x[split_point:batch_size,:,:]
  y_test = y[split_point:batch_size][:]

  return (x_train,y_train,features),(x_test,y_test)



#Gnerate Model For Optimization
def create_autoencoder(train_data,max_filters = 32, min_filters = 8, layers = 3,
                      kernel_size = 3,filter_spacing = 'lin'):
  
  timesteps = 2000
  features = 1
  inputs = Input(shape = (timesteps,features),name = 'Input')
  encoded = inputs
  
  #Linear Filter Range
  if filter_spacing == 'lin':
    filter_range = np.linspace(max_filters,min_filters,layers,
                               dtype = 'int64')
  #Half Filter Range
  if filter_spacing == 'half':
    if max_filters%2 != 0:
      max_filters+=1
    if min_filters%2 != 0:
      min_filters+=1
    filter_range = [max_filters]
    x = max_filters
    while x> min_filters:
      x = x/2
      filter_range.append(x)
    if filter_range[-1] < min_filters:
      del filter_range[-1]
    filter_range = np.asarray(filter_range,dtype = 'int64')
  
  #Random Filter Range
  if filter_spacing == 'random':
    filter_range = [randint(min_filters, max_filters) for z in range(layers)]
    filter_range.sort(reverse= True)
    filter_range = np.asarray(filter_range)
    
  conv_name = ''
  count = 1
  for num_filters in filter_range: 
    conv_name = 'Conv_Layer_No._{0}'.format(count)
    encoded = Conv1D(num_filters,kernel_size,padding = 'same',name = conv_name)(encoded)
    count+=1
    if(num_filters==max_filters):
      encoded = BatchNormalization()(encoded)
  
    encoded = Activation('relu')(encoded)
    encoded = MaxPooling1D(2,padding = 'same')(encoded)
  
  filter_range = np.sort(filter_range)
  for num_filters in filter_range: 
    conv_name = 'Conv_Layer_No._{0}'.format(count)
    encoded = Conv1D(num_filters,kernel_size,padding = 'same',name = conv_name)(encoded)
    count+=1
    encoded = Activation('relu')(encoded)
    encoded = UpSampling1D(2)(encoded)
    
  decoded = Conv1D(1,kernel_size,activation = 'sigmoid',padding = 'same')(encoded)
  
  autoencoder = Model(inputs,decoded)
  autoencoder.summary()
  optimizer = RMSprop(lr=1e-3)
  autoencoder.compile(optimizer = optimizer, loss = 'mse')
  
  return autoencoder


#Sample Model - NOT THE OPTIMIZED MODEL
def create_Conv1D():
  inputs = Input(shape = (timesteps,features))
  
  encoded = Conv1D(32,3, padding = 'same')(inputs)
  encoded = BatchNormalization()(encoded)
  encoded = Activation('relu')(encoded)
  encoded = MaxPooling1D(2,padding = 'same')(encoded)
  
  
  encoded = Conv1D(16,3,padding = 'same')(encoded)
  encoded = Activation('relu')(encoded)
  encoded = MaxPooling1D(2,padding = 'same')(encoded)
  
  encoded = Conv1D(8,3, padding = 'same')(encoded)
  encoded = Activation('relu')(encoded)
  encoded = MaxPooling1D(2,padding = 'same')(encoded)
  
  
  decoded = Conv1D(8,3,padding = 'same')(encoded)
  decoded = Activation('relu')(decoded)
  decoded = UpSampling1D(2)(decoded)
  
  decoded = Conv1D(16,3, padding = 'same')(decoded)
  decoded = Activation('relu')(decoded)
  decoded = UpSampling1D(2)(decoded)
  
  decoded = Conv1D(32,3,padding = 'same')(decoded)
  decoded = Activation('relu')(decoded)
  decoded = UpSampling1D(2)(decoded)
  
  decoded = Conv1D(1,3,activation = 'sigmoid',padding = 'same')(decoded)
  
  autoencoder = Model(inputs,decoded)
  
  optimizer = RMSprop(lr=1e-3)
  autoencoder.compile(optimizer = optimizer, loss = 'mse')
  autoencoder.summary()
  
  return autoencoder


#HyperParameter Range
hyper_params = {
    "data":{
        
        
    },
    "model":{
        "max_filters":hp.choice('mx_filters',np.arange(32,320,dtype = 'int32')),
        "min_filters":hp.choice('mn_filters',np.arange(4,32,dtype = 'int32')),
        'layers':hp.choice('ly',np.arange(3,4,dtype = 'int32')),
        'filter_spacing':hp.choice('fs',['lin','half','random'])
        
    },
    "fit":{
        'epochs': 150,
        'patience': 3
    }
}



#NN Evaluation
db_name = 'sine'
exp_name ='exp_1'
objective = CompileFN(db_name,exp_name,
                      data_fn = create_data,
                      model_fn = create_autoencoder,
                      loss_metric = 'loss',
                      loss_metric_mode = 'min',
                      valid_split = None,
                      save_model = 'best')
#                     save_results = True,
#                     save_dir = model_file.SetContentFile('model.h5')
                      


# model_file.Upload()
# drive.CreateFile({'id': model_file.get('id')})

#test_fn(objective, hyper_params)
trials = Trials()
best = fmin(objective, hyper_params, trials=trials, algo=tpe.suggest, max_evals=2)
print(trials.trials)
#trials = KMongoTrials(db_name, exp_name,
#                      ip="localhost",
#                      port=22334)
#best = fmin(objective, hyper_params, trials=trials, algo=tpe.suggest, max_evals=2)




 
#Save Model to Google Drive
# autoencoder.save('model.h5', include_optimizer=False)
# model_file = drive.CreateFile({'title' : 'CNN1D_v1.0.h5'})
# model_file.SetContentFile('model.h5')
# model_file.Upload()
# drive.CreateFile({'id': model_file.get('id')})
  
  
  
  
  
  

2018-12-25 06:10:55,483 [INFO] tpe_transform took 0.021171 seconds
2018-12-25 06:10:55,485 [INFO] TPE using 0 trials
2018-12-25 06:10:55,491 [INFO] Load data...
2018-12-25 06:10:56,031 [INFO] Fit...


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 2000, 1)           0         
_________________________________________________________________
Conv_Layer_No._1 (Conv1D)    (None, 2000, 287)         1148      
_________________________________________________________________
activation_142 (Activation)  (None, 2000, 287)         0         
_________________________________________________________________
max_pooling1d_81 (MaxPooling (None, 1000, 287)         0         
_________________________________________________________________
Conv_Layer_No._2 (Conv1D)    (None, 1000, 285)         245670    
_________________________________________________________________
activation_143 (Activation)  (None, 1000, 285)         0         
_________________________________________________________________
max_pooling1d_82 (MaxPooling (None, 500, 285)          0         
__________

2018-12-25 06:12:13,578 [INFO] Evaluate...
2018-12-25 06:12:19,942 [INFO] Done!
2018-12-25 06:12:19,973 [INFO] tpe_transform took 0.026975 seconds
2018-12-25 06:12:19,974 [INFO] TPE using 1/1 trials with best loss 0.004169
2018-12-25 06:12:19,984 [INFO] Load data...
2018-12-25 06:12:20,669 [INFO] Fit...


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 2000, 1)           0         
_________________________________________________________________
Conv_Layer_No._1 (Conv1D)    (None, 2000, 106)         424       
_________________________________________________________________
batch_normalization_13 (Batc (None, 2000, 106)         424       
_________________________________________________________________
activation_148 (Activation)  (None, 2000, 106)         0         
_________________________________________________________________
max_pooling1d_84 (MaxPooling (None, 1000, 106)         0         
_________________________________________________________________
Conv_Layer_No._2 (Conv1D)    (None, 1000, 53)          16907     
_________________________________________________________________
activation_149 (Activation)  (None, 1000, 53)          0         
__________

2018-12-25 06:13:13,870 [INFO] Evaluate...
2018-12-25 06:13:20,304 [INFO] Done!
2018-12-25 06:13:20,308 [INFO] PROTOCOL mongo
2018-12-25 06:13:20,310 [INFO] USERNAME None
2018-12-25 06:13:20,312 [INFO] HOSTNAME localhost
2018-12-25 06:13:20,313 [INFO] PORT 22334
2018-12-25 06:13:20,315 [INFO] PATH /sine/jobs
2018-12-25 06:13:20,316 [INFO] AUTH DB None
2018-12-25 06:13:20,318 [INFO] DB sine
2018-12-25 06:13:20,319 [INFO] COLLECTION jobs


[{'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': 0.004169232677668333, 'status': 'ok', 'eval': {'loss': 0.004169232677668333}, 'param': {'data': {}, 'fit': {'epochs': 150, 'patience': 3, 'batch_size': 32, 'early_stop_monitor': 'val_loss'}, 'model': {'filter_spacing': 'random', 'layers': 3, 'max_filters': 311, 'min_filters': 10}}, 'path': {'model': '/root/.kopt/data//sine/exp_1//train_models/9a265851-36bd-48b5-b5ac-48801480dc07.h5', 'results': '/root/.kopt/data//sine/exp_1//train_models/9a265851-36bd-48b5-b5ac-48801480dc07.json'}, 'name': {'data': 'create_data', 'model': 'create_autoencoder', 'optim_metric': 'loss', 'optim_metric_mode': 'loss'}, 'history': {'params': {'batch_size': 32, 'epochs': 150, 'steps': None, 'samples': 850, 'verbose': 2, 'do_validation': True, 'metrics': ['loss', 'val_loss']}, 'loss': {'epoch': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'val_loss': [0.008536544901629289, 0.009959137129286925, 0.006997741473217805, 0.007034423239529133, 0.006434278003871441, 0.004841

ServerSelectionTimeoutError: ignored